<a href="https://colab.research.google.com/github/KawaraAyano/20220228basictask1/blob/main/BERT%E4%BA%8B%E5%89%8D%E5%AD%A6%E7%BF%92%E3%83%A2%E3%83%87%E3%83%AB%E3%82%92%E4%BD%BF%E3%81%A3%E3%81%A6%E3%81%BF%E3%82%88%E3%81%86_ipynb_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

参考参考
https://note.com/npaka/n/n5bb043191cc9
https://qiita.com/karaage0703/items/30485c2ba1c396760982

# 環境構築

In [ ]:
#必要なライブラリのセットアップ
!pip install transformers
!pip install pandas
!pip install ipadic
!pip install fugashi

     |████████████████████████████████| 3.5 MB 7.6 MB/s 
     |████████████████████████████████| 596 kB 46.0 MB/s 
     |████████████████████████████████| 67 kB 5.3 MB/s 
     |████████████████████████████████| 6.8 MB 19.4 MB/s 
     |████████████████████████████████| 895 kB 43.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 13.4 MB 5.7 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=4b60385b8475fd1b63f8f8bec7c79e4a2eed3ceff5f315b95f1a07005caad712
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
Successfully built ipadic
     |████████████████████████████████| 568 kB 9.2 MB/s 


In [ ]:
#MeCabのインストール
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7
!pip install fugashi ipadic

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-470
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl libcgi-pm-perl
  libclass-accessor-perl libcwidget3v5 libencode-l

In [ ]:
#ドライブをcolaboratoryにマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#ライブラリが正しくインストールされているか確認、エラーが出なければエラーが出なければok
import torch
import MeCab

# マスク予測をやってみる

# 下準備

In [ ]:
#対象となる文章
sentence = '私はとても幸せです。'
input_sentence = '[CLS]{}[SEP]'.format(sentence)

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM, BertConfig
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from transformers import BertJapaneseTokenizer, pipeline, BertForSequenceClassification, BertConfig

In [ ]:
model_name = "daigo/bert-base-japanese-sentiment"

#modelの設定の定義
config = BertConfig.from_pretrained( model_name, output_hidden_states=True)

#tokenizerの読み込みと保存 文章をモデルに入力するために整形するツール
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)#download
tokenizer.save_pretrained("./tokenizer")#save
#tokenizerが正常に動くか確認
tokenizer.tokenize('お腹が痛いので遅れます。')

#モデルの読み込みと保存
model = BertForSequenceClassification.from_pretrained(model_name, config = config)#BertForMaskedLM:modeltype
model.save_pretrained('./model')

## 方法1:tokenizer

In [ ]:
##BERT入力前の下準備
#入力文をトークンに分割　単語ごとに元の文章を分割する
tokenized_input = tokenizer.tokenize(input_sentence)
print(tokenized_input)

#トークン化された文章をBERTに入力できる形に変換　数字(ID)に変換
id_input = tokenizer.convert_tokens_to_ids(tokenized_input)
print(id_input)
tensor_input = torch.tensor([id_input])　#tensor型に直す必要がある
print(tensor_input)

#[MASK]のトークンのidを取得　　
mask_token_index = torch.where(tensor_input == tokenizer.mask_token_id)[1]

['[CLS]', '私', 'は', '明日', 'の', '午後', '近所', 'の', '海', '##鮮', '丼', '屋', 'で', '[MASK]', 'を', '食べる', '予定', 'だ', '。', '[SEP]']
[2, 1325, 9, 11475, 5, 4776, 16703, 5, 295, 29483, 21235, 1341, 12, 4, 11, 6303, 1484, 75, 8, 3]
tensor([[    2,  1325,     9, 11475,     5,  4776, 16703,     5,   295, 29483,
         21235,  1341,    12,     4,    11,  6303,  1484,    75,     8,     3]])


In [ ]:
show_top = 5 #予測の上位何件を表示するか
with torch.no_grad(): #推論のときは必ず付ける 学習とは違う(logitではなくloss誤差になる)
  token_logits = model(tensor_input).logits
  print(token_logits.size())
  mask_token_logits = token_logits[0, mask_token_index, :]
  print(mask_token_logits.size())#maskpartのみ取り出す
  top_tokens = torch.topk(mask_token_logits, show_top, dim=1).indices[0].tolist()#ID
  top_values = torch.topk(mask_token_logits, show_top, dim=1).values[0].tolist()#確信度
  print(top_tokens)
  print(top_values)
for i in range(show_top):#順番に並べる
  token = tokenizer.decode(top_tokens[i])#IDから日本語に変換
  value = top_values[i]
  print('token: {}, confidence: {}'.format(token, value))


torch.Size([1, 20, 32000])
torch.Size([1, 32000])
[27073, 2171, 27966, 25965, 12396]
[11.494487762451172, 11.186466217041016, 11.118791580200195, 11.09034252166748, 10.800607681274414]
token: ご 飯, confidence: 11.494487762451172
token: 魚, confidence: 11.186466217041016
token: 夕 食, confidence: 11.118791580200195
token: 朝 食, confidence: 11.09034252166748
token: カ レ ー, confidence: 10.800607681274414


## 方法2:pipeline

In [ ]:
from transformers import pipeline

#パイプラインの作成
nlp = pipeline('sentiment-analysis', model = model, tokenizer = tokenizer)
#fill-maskやりたいtaskに合わせて変える

#mask predictionの実行
result = nlp(input_sentence)

#結果の出力
for r in result:
  print(r)

{'label': 'ポジティブ', 'score': 0.9912922978401184}
